# Reflection


In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions.
This is then used downstream for things like re-planning, search, or evaluation.

![Reflection](./img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.

#### Prerequisites

We will be using a basic agent with a search tool here.

## Setup

### Load env vars

Add a `.env` variable in the root of the `./examples` folder with your variables.

In [4]:
import { load } from "https://deno.land/std@0.215.0/dotenv/mod.ts";

const env = await load({ envPath: "../.env" });
Object.entries(env).forEach(([key, value]) => Deno.env.set(key, value));

## Generate

For our example, we will create a "5 paragraph essay" generator. First, create the generator:

In [5]:
import { ChatFireworks } from "@langchain/community/chat_models/fireworks";
import { BaseMessage, AIMessage, HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", `You are an essay assistant tasked with writing excellent 5-paragraph essays.
Generate the best essay possible for the user's request.
If the user provides critique, respond with a revised version of your previous attempts.`],
  new MessagesPlaceholder("messages")
]);
const llm = new ChatFireworks({
  modelName: "accounts/fireworks/models/mixtral-8x7b-instruct",
  temperature: 0,
  modelKwargs: {
    max_tokens: 32768
  }
});
const generate = prompt.pipe(llm)

In [6]:
let essay = "";
const request = new HumanMessage({
  content: "Write an essay on why the little prince is relevant in modern childhood"
});

for await (const chunk of await generate.stream({ messages: [request] })) {
  console.log(chunk.content);
  essay += chunk.content;
}


Title:
 The Relevance of The Little Prince in Modern
 Childhood

Introduction:
Antoine de
 Saint-Exupéry's The Little
 Prince is a timeless
 novella that has captured the hearts and minds of
 both children and adults for generations. Despite being published
 over seven decades ago, the story remains relevant in
 modern childhood for its profound themes and life lessons.
 This essay will explore the reasons why The Little
 Prince continues to resonate with children today.


Body Paragraph 1: Relatability
 of Characters and Emotions
The Little
 Prince's characters and their emotions are remarkably
 relatable to children in the modern world. The
 protagonist, a young prince from asteroid B
-612, embodies curiosity, innoc
ence, and a strong sense of morality.
 These traits are
 universally cherished and aspired to by children
 and adults alike. Additionally, the various inhabitants of
 the planets the Little Prince visits, such as the
 conceited man, the drunkard, and the
 businessman, se

"Title: The Relevance of The Little Prince in Modern Childhood\n" +
  "\n" +
  "Introduction:\n" +
  "Antoine de Saint-Exupér"... 6356 more characters

### Reflect

In [8]:
const reflectionPrompt = ChatPromptTemplate.fromMessages([
  ["system", `You are a teacher grading an essay submission.
Generate critique and recommendations for the user's submission.
Provide detailed recommendations, including requests for length, depth, style, etc.`],
new MessagesPlaceholder("messages"),
])
const reflect = reflectionPrompt.pipe(llm)

In [9]:
let reflection = "";

for await (const chunk of await reflect.stream({ messages: [request, new HumanMessage({ content: essay })] })) {
  console.log(chunk.content);
  reflection += chunk.content;
}


Dear student
,

I would like to commend you
 on your thoughtful and
 well-structured essay on the relevance of
 The Little Prince in modern childhood. Your analysis of
 the characters, themes, and messages conveyed in
 the novella is insightful
 and shows a strong understanding of the material.
 However, there are a few areas where your essay
 could be improved to better convey your ideas and engage
 the reader.

1.
 Essay Length: Your
 essay is well within the required word count,
 but I would suggest expanding upon your ideas to create
 a more comprehensive analysis. Adding more specific examples
 from the text and fleshing out your arguments
 will strengthen your essay and make it more engaging for
 the reader.

2. Depth:
 While your essay touches on various aspects of the nov
ella, I would like to see you delve
 deeper into the themes and messages. For instance,
 you could further explore how the themes
 of loneliness, responsibility, and the search
 for meaning are reflected in t

"Dear student,\n" +
  "\n" +
  "I would like to commend you on your thoughtful and well-structured essay on the relev"... 2552 more characters

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [11]:
for await (const chunk of await generate.stream({ messages: [request, new AIMessage({ content: essay }), new HumanMessage({ content: reflection })] })) {
  console.log(chunk.content);
}


Dear [
Your Name],

Thank you for your thought
ful and constructive feedback on my essay about the
 relevance of The Little Prince in modern childhood.
 I appreciate the time you took to provide detailed suggestions
 for improvement, and I am eager to revise
 my work to better convey
 my ideas and engage the reader.

1
. Essay Length: I will expand upon my
 ideas and provide more specific examples from the text to
 create a more comprehensive analysis. This will allow me
 to delve deeper into the themes and messages of
 the novella and strengthen my arguments.
2
. Depth: I will further explore how the
 themes of loneliness, responsibility, and the
 search for meaning are reflected in the modern world and
 how they impact children's lives. By providing
 more context and real-world examples, I aim
 to solidify my arguments and demonstrate the relevance
 of these themes in contemporary society.
3.
 Style: I will incorporate more varied sentence structures and
 vocabulary to make my essay

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.

In [17]:
import { END, MessageGraph } from "@langchain/langgraph";
import { RunnableConfig } from "@langchain/core/runnables";

const generationNode = async (messages: BaseMessage[], config?: RunnableConfig) => {
  return [await generate.invoke({ messages }, config)];
}

const reflectionNode = async (messages: BaseMessage[], config?: RunnableConfig) => {
    // Other messages we need to adjust
    const clsMap: { [key: string]: new (content: string) => BaseMessage } = {
      "ai": HumanMessage,
      "human": AIMessage,
  };
  // First message is the original user request. We hold it the same for all nodes
  const translated = [messages[0], ...messages.slice(1).map((msg) => new clsMap[msg._getType()](msg.content))];
  const res = await reflect.invoke({ "messages": translated }, config);
  // We treat the output of this as human feedback for the generator
  return [new HumanMessage(res.content)];
}

// Define the graph
const workflow = new MessageGraph();
workflow.addNode("generate", generationNode);
workflow.addNode("reflect", reflectionNode);
workflow.setEntryPoint("generate");

const shouldContinue = (messages: BaseMessage[]) => {
  if (messages.length > 6) {
    // End state after 3 iterations
    return END;
  }
  return "reflect";
}

workflow.addConditionalEdges("generate", shouldContinue);
workflow.addEdge("reflect", "generate");

const app = workflow.compile();

In [18]:
let finalRes;

for await (const event of await app.stream([new HumanMessage({
  content: "Generate an essay on the topicality of The Little Prince and its message in modern life"
})])) {
  finalRes = event;
  for (const [key, value] of Object.entries(event)) {
    console.log(`Event: ${key}`);
    // Uncomment to see the result of each step.
    // console.log(value.map((msg) => msg.content).join("\n"));
    console.log("\n------\n")
  }
}

Event: generate

------

Event: reflect

------

Event: generate

------

Event: reflect

------

Event: generate

------

Event: reflect

------

Event: generate

------

Event: __end__

------



In [20]:
console.log(finalRes[END].map((msg) => msg.content).join("\n\n\n------------------\n\n\n"));

Generate an essay on the topicality of The Little Prince and its message in modern life


------------------


Title: The Little Prince: A Timeless Message for Modern Life

Introduction:
Antoine de Saint-Exupéry's The Little Prince is a classic novella that has captured the hearts of millions of readers worldwide since its publication in 1943. The story, which follows a young prince traveling through various planets, has been translated into over 300 languages and continues to resonate with people of all ages and backgrounds. This essay will explore the topicality of The Little Prince and its enduring message in modern life.

Body Paragraph 1: The Relevance of The Little Prince's Themes

The Little Prince touches on several themes that remain relevant in modern life, such as the importance of human connections, the dangers of materialism, and the value of looking beyond superficial appearances. The story's exploration of these themes is particularly resonant in today's fast-paced, tech

> #### See the LangSmith trace [here](https://smith.langchain.com/public/47326333-c98f-44ae-9323-f9d191b74469/r)